In [18]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import os
from bardapi import Bard

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = ""

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)


def get_comments(video_id, page_token=None):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        pageToken=page_token
    )
    response = request.execute()

    comments = []
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([
            comment['authorDisplayName'],
            comment['publishedAt'],
            comment['updatedAt'],
            comment['likeCount'],
            comment['textDisplay']
        ])

    if 'nextPageToken' in response:
        next_page_comments = get_comments(video_id, response['nextPageToken'])
        comments.extend(next_page_comments)

    return comments


# Get all comments for the video with ID "0DlSRxEq0W4"
comments = get_comments("cT93a0ENcqg")

# Save the comments to a DataFrame
df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])


In [19]:
df.head(30000)

,author,published_at,updated_at,like_count,text
0,FukGoogle AndGayusa,2023-11-03T22:41:12Z,2023-11-03T22:41:12Z,0,Free Palestine 🇵🇸
1,FukGoogle AndGayusa,2023-11-03T22:39:03Z,2023-11-03T22:39:03Z,0,They brought it to themselves they should bla...
2,Loriah Izimrat,2023-11-03T08:52:06Z,2023-11-03T08:52:06Z,0,The world need trump to make peace
3,Masroor Ahmad,2023-11-03T05:05:27Z,2023-11-03T05:05:27Z,0,"Background and facts:\r<br>➡ After the Nakba, ..."
4,Masroor Ahmad,2023-11-03T05:04:03Z,2023-11-03T05:04:03Z,0,"During the Oslo, Israel gained control over 60..."
...,...,...,...,...,...
1864,PD,2023-10-11T15:35:06Z,2023-10-11T15:35:06Z,0,I am in UK 🇬🇧 and confirm that the so called G...
1865,PD,2023-10-11T15:30:39Z,2023-10-11T15:30:39Z,0,Waiting 👍
1866,Mohit Joshi,2023-10-11T15:28:05Z,2023-10-11T15:34:44Z,170,"At this point, Bharat seems to be the only maj..."
1867,༒•Tsar•༒,2023-10-11T15:23:19Z,2023-10-11T16:04:47Z,13,Everything in the World is about Modi ❤<br><br...


In [20]:
comments = df["text"]

In [21]:
comments.head(6)

0                                    Free Palestine 🇵🇸
1    They brought it to themselves they  should bla...
2                   The world need trump to make peace
3    Background and facts:\r<br>➡ After the Nakba, ...
4    During the Oslo, Israel gained control over 60...
5    Great! But, you did not touch up on how Israel...
Name: text, dtype: object

In [22]:
prompt =""
for i in range(30):
    prompt = prompt+str(i+1)+". "+comments[i]


In [23]:
text=comments.to_string()

In [24]:
text

'0                                       Free Palestine 🇵🇸\n1       They brought it to themselves they  should bla...\n2                      The world need trump to make peace\n3       Background and facts:\\r<br>➡ After the Nakba, ...\n4       During the Oslo, Israel gained control over 60...\n5       Great! But, you did not touch up on how Israel...\n6       &quot;For a change, US was trying to actually ...\n7                                    So nicely explained\U0001f979\n8       How biased is Think School, when telling the h...\n9       Isra3l People India Love You.. 🇮🇳❤️🇮🇱 You love...\n10      As usual Indian perspective, ignorant, lacking...\n11      You forgot to mention nearly $300 billion trad...\n12      Trade route being built from India through the...\n13      Hy Ganesh sir, In every election you made vide...\n14      Correction, Israel only wants peace if they ca...\n15      Correction US intentions is not about making p...\n16      Correction, Iran is not the main cont

In [25]:
Question = "Give an overall review as a third person in very small paragraph, considering all the comments so that it would useful for the content creator to have the review on the good and bad things about the video.The comments are "+prompt

os.environ["_BARD_API_KEY"] = ""
a = Bard().get_answer(str(Question))['content']
print(a)

Exception: SNlM0e value not found. Double-check __Secure-1PSID value or pass it as token='xxxxx'.